# Implementing flow

Notebook for implementing the Flow algorithm in the Attention structure.

Main idea: Given a model, assign as **source** its **class token** embedding at **final layer**. The **sinks** will be the **input tokens**. The Graph will be generated by the matrices $A$ as in the paper:

$$
A = \frac{W_{att} + I}{2} 
$$

Plus we have to renormalize (check how normalization is done)

In [1]:
import torch
from PIL import Image
import numpy
import sys
from torchvision import transforms
import numpy as np
import cv2

import maxflow

/Data/ml_env/miniconda3/envs/venv_ml_env/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Data/ml_env/miniconda3/envs/venv_ml_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def compute_a_matrices(attentions, discard_ratio, head_fusion):
    '''Generates the A matrices as in the paper from the attention layers
    '''
    a_matrices = []
    
    for attention in attentions:
        # Getting type of fusion of channels
        if head_fusion == "mean":
            attention_heads_fused = attention.mean(axis=1)
        elif head_fusion == "max":
            attention_heads_fused = attention.max(axis=1)[0]
        elif head_fusion == "min":
            attention_heads_fused = attention.min(axis=1)[0]
        else:
            raise "Attention head fusion type Not supported"
        
        # Drop the lowest attentions, but
        # don't drop the class token
        flat = attention_heads_fused.view(attention_heads_fused.size(0), -1)
        _, indices = flat.topk(int(flat.size(-1)*discard_ratio), -1, False)
        indices = indices[indices != 0]
        flat[0, indices] = 0
        
        
        I = torch.eye(attention_heads_fused.size(-1))
        a = (attention_heads_fused + 1.0*I)/2
        
        a = a / a.sum(dim=-1)
        # a = a / a.sum(dim=-1, keepdim=True) -> verify which one is correct
        
        a_matrices.append(a)  
    
    return a_matrices
    
    
def compute_flow(a_matrices, input_node, output_flow, discard_ratio):
    '''Compute flow of a single input source
    '''
    n_tokens = a_matrices[0].size(-1)
    n_layers = len(a_matrices)
    
    n_nodes = n_layers * n_tokens

    n_vertices = int((1 - discard_ratio) * n_tokens**2 + 1)

    print(n_nodes)
    print(n_vertices)

    g = maxflow.Graph[float](n_nodes, n_vertices) 
    nodes = g.add_nodes(n_nodes) 

    ## Setting first nodes who will be sink

    for i in range(n_tokens):
        g.add_tedge(nodes[i], 0., output_flow)
        
    ## Setting final nodes who will be source
    source_weights = a_matrices[-1][0, :, input_node]
    for idx, node_number in enumerate(range(n_nodes - n_tokens, n_nodes)):
        g.add_tedge(nodes[node_number], source_weights[idx], 0)

    ## Setting internal nodes

    for n_layer, a_matrix in enumerate(a_matrices):
        if n_layer == len(a_matrices) - 1: break
        
        start_node = n_layer * n_tokens
        start_node_next = (n_layer + 1) * n_tokens
        
        for idx_x, node_number in enumerate(range(start_node, start_node + n_tokens)):
            weights = a_matrix[0,idx_x,:]
            
            for idx_y in range(n_tokens):
                if idx_y != 0: continue
                weight = weights[idx_y]
                
                if weight == 0: continue
                
                node_number_next = start_node_next + idx_y
                g.add_edge(nodes[node_number], nodes[node_number_next], 0., weight) # next layer points to layer before
                
    max_flow = g.maxflow()
    
    return max_flow


def compute_all_flows(a_matrices, output_flow, discard_ratio):
    '''Compute flow for all sources
    '''
    n_tokens = a_matrices[0].size(-1)
    
    mask = torch.Tensor(np.zeros(n_tokens))
    
    for n_token in range(n_tokens):
        mask[n_token] = compute_flow(a_matrices, n_token, output_flow, discard_ratio)
        
    mask = mask[1:]
    
    width = int(mask.size(-1)**0.5)
    mask = mask.reshape(width, width).numpy()
    mask = mask / np.max(mask)
    
    return mask

def flow(attentions, discard_ratio, head_fusion, output_flow=2.):
    '''Generates attention flow mask in similar fashion as rollout
    '''
    
    a_matrices = compute_a_matrices(attentions, discard_ratio, head_fusion)
    
    # must generate mask
    mask = compute_all_flows(a_matrices, output_flow, discard_ratio)

    # Adjust for output
    width = int(mask.size(-1)**0.5)
    mask = mask.reshape(width, width).numpy()
    mask = mask / np.max(mask)
    
    return mask

class VITAttentionFlow:
    def __init__(self, model, attention_layer_name='attn_drop', head_fusion="mean", discard_ratio=0.9):
        self.model = model
        self.head_fusion = head_fusion
        self.discard_ratio = discard_ratio
        
        for name, module in self.model.named_modules():
            if attention_layer_name in name:
                module.register_forward_hook(self.get_attention)

        self.attentions = []

    def get_attention(self, module, input, output):
        self.attentions.append(output.cpu())

    def __call__(self, input_tensor):
        self.attentions = []
        with torch.no_grad():
            output = self.model(input_tensor)

        return flow(self.attentions, self.discard_ratio, self.head_fusion)